In [1]:
from tensorflow.keras.models import load_model ,Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Flatten , Dropout , Dense
from tensorflow.keras.preprocessing.image import img_to_array , load_img
import cv2 
import numpy as np

In [2]:
vgg = VGG16(weights='./models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',include_top=False , input_shape=(224,224,3))  

In [3]:
flatten = vgg.output
flatten = Flatten()(flatten)


bboxHead = Dense(128, activation='relu')(flatten)
bboxHead = Dense(64, activation='relu')(bboxHead)
bboxHead = Dense(32, activation='relu')(bboxHead)
bboxHead = Dense(4, activation='sigmoid' , name = "bounding_box")(bboxHead)



softmaxHead = Dense(512,activation = 'relu')(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512,activation = 'relu')(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(8,activation = 'softmax',name = "class_label")(softmaxHead)



model = Model(inputs=vgg.input , outputs = (softmaxHead , bboxHead))



In [4]:
model.load_weights('weght.h5')

In [5]:
def draw_bbox( img , box , lbl ):
    image = np.copy(img)
    (h,w) = image.shape[:2]
    
    (X , Y , dX , dY) = box[0]
    
    
    X = X*w
    Y = Y*h
    
    dX = dX * w
    dY = dY * h
    
    startX =int( X - dX/2 )
    startY =int( Y - dY/2 )
    
    endX = int ( X + dX/2 )
    endY = int ( Y + dY/2 )
    
    y = startY - 10 if startY - 10 > 10 else startY + 10 
    
    cv2.putText(image , lbl , (startX , y) ,cv2.FONT_HERSHEY_SIMPLEX ,0.65 ,(0,0,0) ,2)
    cv2.rectangle(image, (startX , startY) , (endX , endY) ,(0,0,0) ,2)
    
    return image
     
    

In [6]:
decode_predictions = { 0:'Zero' ,  1:'Dislike' ,  2:'Five' ,  3:'Exactly' ,  4:'Two' ,  5:'Three' ,  6:'Left' ,  7:'Like' }


In [7]:
cap = cv2.VideoCapture(0)

while True :
    ret , frame = cap.read()
    frame_copy = cv2.resize(frame , (224,224))/255.0
    inputs = np.expand_dims(frame_copy[:,:,::-1],axis = 0)
    lbl , box = model.predict(inputs)
    lbl = decode_predictions[np.argmax(lbl)]
    img = draw_bbox(frame ,box , lbl)
    cv2.imshow('Webcam' , img)
    if cv2.waitKey(1) == 13 :
        break
 
 
 
 
cv2.destroyAllWindows()
cap.release()    
    

1/1 [==============================] - 0s 137ms/step
